In [1]:
import boto3

In [2]:
s3 = boto3.client('s3', region_name='ap-south-1')

In [3]:
response = s3.list_buckets()

In [4]:
response

{'ResponseMetadata': {'RequestId': '3PC6KZA6F0T100WD',
  'HostId': 'oEmrLtkHJL9l7EKNeGQqLyKjgaYw6I9nkJql+7xJT9a2dBA3m6LuHiC/atqcxyu/5DUZ894GvC0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'oEmrLtkHJL9l7EKNeGQqLyKjgaYw6I9nkJql+7xJT9a2dBA3m6LuHiC/atqcxyu/5DUZ894GvC0=',
   'x-amz-request-id': '3PC6KZA6F0T100WD',
   'date': 'Wed, 27 Nov 2024 08:13:22 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'sagemaker-ap-south-1-065936145461',
   'CreationDate': datetime.datetime(2024, 11, 8, 3, 57, 53, tzinfo=tzutc())},
  {'Name': 'sagemaker-studio-065936145461-ckmixblb6z',
   'CreationDate': datetime.datetime(2024, 11, 13, 17, 37, 10, tzinfo=tzutc())},
  {'Name': 'sagemaker-studio-065936145461-zfar3dmzei',
   'CreationDate': datetime.datetime(2024, 7, 28, 2, 9, 35, tzinfo=tzutc())},
  {'Name': 'sagemaker-us-east-1-065936145461',
   'CreationDate': datetime.datetime(2024, 7, 28, 2, 

In [5]:
def create_bucket(bucket_name, region='ap-south-1'): 
    try:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
        )
        print(f"Bucket '{bucket_name}' created successfully in region '{region}'.")
    except Exception as e:
        print(f"Error: {e}")

### AWS provides naming standards when naming an aws bucket.

###### The bucket name can be between 3 and 63 characters long, and can contain only lower-case characters, numbers, periods, and dashes.
###### Each label in the bucket name must start with a lowercase letter or number.
###### The bucket name cannot contain underscores, end with a dash, have consecutive periods, or use dashes adjacent to periods.
###### The bucket name cannot be formatted as an IP address (198.51.100.24).
###### The name provided contains upper case letters, by switching to lower case letters the issue can be resolved.

In [6]:
create_bucket('s3agraaws', region='ap-south-1')

Bucket 's3agraaws' created successfully in region 'ap-south-1'.


### Uploading Single File On S3 Bucket

In [7]:
import os
def upload_file(file_path, bucket_name, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_path)
    
    s3.upload_file(file_path,bucket_name,object_name)

In [9]:
upload_file(file_path = 'AGRADWIP KARMAKAR.pdf', bucket_name='s3agraaws',object_name='AK.pdf')

In [10]:
upload_file(file_path = 'data/index.html', bucket_name='s3agraaws',object_name='index.html')

### List Out All Objects In The Bucket

In [13]:
bucket_name = 's3agraaws'

In [14]:
def list_objects(bucket_name):
    response = s3.list_objects_v2(Bucket = bucket_name)
    for obj in response['Contents']:
        print(obj['Key'])

In [15]:
list_objects(bucket_name)

AK.pdf
index.html


### Douwnload Objects From S3 Bucket

In [16]:
def download_file(bucket_name, object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    s3.download_file(bucket_name, object_name, file_path)

In [17]:
download_file(bucket_name=bucket_name, object_name='index.html', file_path='data_download/index.html')

### Upload A Whole Folder

In [18]:
for root, dirs, files in os.walk('data'):
    print(root, dirs, files)

data [] ['index.html', 'panda-1236875_640.jpg', 'water-5393919_1280.jpg']


In [ ]:
def upload_directory(directory_path, s3_prefix, bucket_name):
    for root, dirs, files in os.walk('data'):
        for file in files:
            file_path = os.path.join(root, file).replace("\\", "/")
            relpath = os.path.relpath(file_path, directory_path)
            s3_key = os.path.join(s3_prefix, relpath).replace("\\", "/")
            s3.upload_file(file_path, bucket_name, s3_key)
            print(f'The file name {file_path} in folder {root} is uploaded on S3 bucket named as {bucket_name}') 

In [26]:
upload_directory('data', 'S3_data',bucket_name)

The file name data/index.html in folder data is uploaded on S3 bucket named as s3agraaws
The file name data/panda-1236875_640.jpg in folder data is uploaded on S3 bucket named as s3agraaws
The file name data/water-5393919_1280.jpg in folder data is uploaded on S3 bucket named as s3agraaws


### Download load A Whole Folder From S3

In [35]:
def download_folder(local_path, s3_prefix, bucket_name):
    os.makedirs(local_path, exist_ok=True)

    paginator = s3.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket = bucket_name, Prefix = s3_prefix):
        if 'Contents' in result:
            for Key in result['Contents']:
                s3_key = Key['Key']

                local_file = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
                os.makedirs(os.path.dirname(local_file), exist_ok=True)

                s3.download_file(bucket_name,s3_key,local_file)

In [36]:
local_path = 'S3_download'
s3_prefix = 'S3_data'
download_folder(local_path,s3_prefix,bucket_name)

### Delete All Files in a Bucket

In [38]:
def delete_objects():
    response = s3.list_objects_v2(Bucket = bucket_name)

    if 'Contents' in response:
        for obj in response['Contents']:
            s3.delete_object(Bucket = bucket_name, Key = obj['Key'])

delete_objects()

In [39]:
s3.delete_bucket(Bucket = bucket_name)

{'ResponseMetadata': {'RequestId': 'MJ79RM0AP6M8YHWN',
  'HostId': 'rOVvJ4/GAzVl7EYFSd9rzAtM21374Qc9buw8Il4Tcpnv+hXw8AgFhYL+bVes8L1B5/E6ZE8PFIXvooi2e1G/Qw==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'rOVvJ4/GAzVl7EYFSd9rzAtM21374Qc9buw8Il4Tcpnv+hXw8AgFhYL+bVes8L1B5/E6ZE8PFIXvooi2e1G/Qw==',
   'x-amz-request-id': 'MJ79RM0AP6M8YHWN',
   'date': 'Wed, 27 Nov 2024 10:44:33 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}